In [1]:
%%configure -f
{
    "driverMemory":"4G",
    "executorMemory":"20G",
    "executorCores":8,
    "numExecutors":10
}

In [2]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
21,application_1625762687750_0088,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from datetime import datetime, timedelta


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
date='2021-06-18'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
model_date = (datetime.strptime(date, "%Y-%m-%d") - timedelta(1)).strftime('%Y-%m-%d')
model_date

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2021-06-17'

In [6]:
outpath='s3a://pai-gm-midgar-shared-ml-team/pengcheng/dampen_intent/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# model ranking

In [29]:
def takeSecond(elem):
    return elem[1]

def _sort(x, n):
    x.sort(key=takeSecond, reverse=True)
    return [x[i][0] for i in range(n)]

schema = ArrayType(StructType([
    StructField("topn", ArrayType(StringType(), False), False)
]))

def _get_top_n_udf(n):
    return F.udf(lambda x: _sort(x, n), ArrayType(StringType(), False))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
def load_model_ranking(date, n=10):
    path='s3a://pai-gm-midgar-production-ml-jobs/shinra/modelling/weighted_trendfiltering_banner_events/user_categories/'
    #spark.read.parquet(path+f'/model_date={date}').printSchema()
    df=spark.read.parquet(path+f'/model_date={date}')
    df2=df.withColumn('topn', _get_top_n_udf(n)('category_trend_scores')).select('customer_id', 'topn')
    return df2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
model_df=load_model_ranking(model_date)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
model_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: long (nullable = true)
 |-- topn: array (nullable = true)
 |    |-- element: string (containsNull = false)

In [11]:
model_df.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------------------------------------------------+
|customer_id|topn                                                      |
+-----------+----------------------------------------------------------+
|5336       |[6802, 10, 6843, 443, 6841, 6797, 6821, 6842, 59, 6826]   |
|7427       |[6842, 6797, 6802, 6843, 444, 6841, 443, 29, 6851, 14]    |
|13326      |[10, 6843, 443, 6841, 6797, 6821, 6842, 59, 6826, 6851]   |
|28455      |[6841, 6843, 443, 6797, 6821, 6842, 10, 59, 6826, 6851]   |
|34132      |[6797, 6843, 6826, 6841, 6852, 6795, 445, 443, 6851, 6842]|
+-----------+----------------------------------------------------------+
only showing top 5 rows

# l1 mapping

In [7]:
mps=spark.read.csv('s3a://midgar-aws-workspace/prod/shinra/base/l1_cma_name_to_id_map/2021-06-26')\
.select(F.col('_c0').alias('l1_category_cma_name'), F.col('_c1').alias('l1_category_cma_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
spark.read.csv('s3a://midgar-aws-workspace/prod/shinra/base/l1_cma_name_to_id_map/2021-06-26').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

123

In [8]:
mps.filter(F.col('l1_category_cma_id')==1).show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|l1_category_cma_name|l1_category_cma_id|
+--------------------+------------------+
|PAYTM               |1                 |
+--------------------+------------------+

In [9]:
mps.filter(F.col('l1_category_cma_id')==5).show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|l1_category_cma_name|l1_category_cma_id|
+--------------------+------------------+
|PAYTM-Bank          |5                 |
+--------------------+------------------+

In [10]:
mps.filter(F.col('l1_category_cma_id')==20).show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|l1_category_cma_name|l1_category_cma_id|
+--------------------+------------------+
|EC-SA               |20                |
+--------------------+------------------+

In [11]:
mps.filter(F.col('l1_category_cma_id')==29).show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|l1_category_cma_name|l1_category_cma_id|
+--------------------+------------------+
|EC-Others           |29                |
+--------------------+------------------+

In [31]:
phaseRules_path = "s3a://midgar-aws-workspace/prod/mapfeatures/lifecycle/tenant/paytm-india/resources/phaseRules"

def get_cma_mapping (phaseRules_path):


    last_date_phaseRules = (spark.read.option("header", "true").csv(phaseRules_path)
                                     .agg(F.max('dt'))
                                     .distinct()
                                     .collect()[0][0]
                                     .strftime("%Y-%m-%d"))


    cma_mapping = (spark.read.option("header", "true").csv(phaseRules_path)
                   .filter(F.col("dt")==last_date_phaseRules)
                  .select("l1_category_cma_id","l1_category_cma_name"))

    return cma_mapping

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
cma_mapping=get_cma_mapping(phaseRules_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
cma_mapping.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+
|l1_category_cma_id|l1_category_cma_name|
+------------------+--------------------+
|1                 |PAYTM               |
|9                 |PAYTM-Money         |
|10                |PAYTM-Games         |
|12                |PAYTM-Gold          |
|14                |EC-Apparel          |
+------------------+--------------------+
only showing top 5 rows

# banner events

In [34]:
def load_banner_events(date):
    bpath="s3a://midgar-aws-workspace/prod/shinra/telescope/dashboard/banner_campaigns/customer_level/banner_campaign_base_customer_level_full/"
    cols=['customer_id', 'l1_category_name', 'date', 'impressions_adjust', 'clicks_adjust', 'category_sales_adjust']
    df=spark.read.parquet(bpath+f'/dt={date}').select(cols).fillna(0)
    df=df.join(cma_mapping, df.l1_category_name==cma_mapping.l1_category_cma_name).drop('l1_category_name')\
         .groupBy('customer_id', 'l1_category_cma_id').agg(F.sum('impressions_adjust').alias('impressions_adjust'),
                                                 F.sum('clicks_adjust').alias('clicks_adjust'),
                                                  F.sum('category_sales_adjust').alias('category_sales_adjust'))
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
banner_df=load_banner_events(date)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
banner_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: string (nullable = true)
 |-- l1_category_cma_id: string (nullable = true)
 |-- impressions_adjust: double (nullable = true)
 |-- clicks_adjust: double (nullable = true)
 |-- category_sales_adjust: double (nullable = true)

In [18]:
banner_df.show(2,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+------------------+-------------+---------------------+
|customer_id|l1_category_cma_id|impressions_adjust|clicks_adjust|category_sales_adjust|
+-----------+------------------+------------------+-------------+---------------------+
|1286373187 |38                |2.0               |0.0          |0.0                  |
|517247594  |20                |0.2857142857142857|0.0          |0.0                  |
+-----------+------------------+------------------+-------------+---------------------+
only showing top 2 rows

# join banner with model

In [35]:
def _rank(x, y):
    for i in range(len(y)):
        if y[i]==x:
            return i+1
    return len(y)+1

_get_rank=F.udf(_rank, IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
def join_df(date):
    model_date = (datetime.strptime(date, "%Y-%m-%d") - timedelta(1)).strftime('%Y-%m-%d')
    model_df=load_model_ranking(model_date)
    banner_df=load_banner_events(date)
    join_df=banner_df.join(model_df, on=['customer_id'])
    
    join_df=join_df.withColumn('rank', _get_rank('l1_category_cma_id', 'topn'))
    join_df.coalesce(200).write.mode('overwrite').parquet(outpath+f"/ctr_category_base/dt={date}")
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
base = datetime.strptime('2021-06-01', "%Y-%m-%d")
date_list = [(base + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(10)]
date_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04', '2021-06-05', '2021-06-06', '2021-06-07', '2021-06-08', '2021-06-09', '2021-06-10']

In [38]:
for d in date_list[:3]:
    join_df(d)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
for d in date_list[3:5]:
    join_df(d)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
for d in date_list[5:7]:
    join_df(d)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
join_df(date_list[7])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
join_df(date_list[8])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
join_df(date_list[9])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# analysis

In [7]:
df=spark.read.parquet(outpath+f"/ctr_category_base/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.select('dt').distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10

In [9]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: string (nullable = true)
 |-- l1_category_cma_id: string (nullable = true)
 |-- impressions_adjust: double (nullable = true)
 |-- clicks_adjust: double (nullable = true)
 |-- category_sales_adjust: double (nullable = true)
 |-- topn: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rank: integer (nullable = true)
 |-- dt: date (nullable = true)

In [10]:
df.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+------------------+-------------+---------------------+-------------------------------------------------------+----+----------+
|customer_id|l1_category_cma_id|impressions_adjust|clicks_adjust|category_sales_adjust|topn                                                   |rank|dt        |
+-----------+------------------+------------------+-------------+---------------------+-------------------------------------------------------+----+----------+
|5101       |6852              |6.0               |0.0          |0.0                  |[6841, 6843, 6797, 6802, 6852, 443, 40, 38, 6842, 6821]|5   |2021-06-09|
|5101       |20                |1.0               |0.0          |0.0                  |[6841, 6843, 6797, 6802, 6852, 443, 40, 38, 6842, 6821]|11  |2021-06-09|
|5101       |9                 |1.0               |0.0          |0.0                  |[6841, 6843, 6797, 6802, 6852, 443, 40, 38, 6842, 6821]|11  |2021-06-09|
|5101       |38                |3.0     

In [24]:
windowSpec  = Window.partitionBy('l1_category_cma_id')
df1=df.groupBy('l1_category_cma_id', 'rank')\
        .agg(F.sum('impressions_adjust').alias('impressions_total'),
             F.sum('clicks_adjust').alias('clicks_total'),
             F.sum('category_sales_adjust').alias('category_sales_total'))\
        .withColumn('ctr', F.col('clicks_total')/F.col('impressions_total'))\
        .withColumn('cvr', F.col('category_sales_total')/F.col('impressions_total'))\
        .withColumn('impression_rate', F.col('impressions_total')/F.sum('impressions_total').over(windowSpec))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df1.show(2,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----+-----------------+------------+--------------------+-------------------+--------------------+--------------------+
|l1_category_cma_id|rank|impressions_total|clicks_total|category_sales_total|ctr                |cvr                 |impression_rate     |
+------------------+----+-----------------+------------+--------------------+-------------------+--------------------+--------------------+
|443               |6   |1.7530262E7      |936107.0    |323323.8200000003   |0.05339948712688949|0.01844375286575867 |0.050931325656874124|
|443               |8   |6456885.0        |304968.0    |92363.60999999997   |0.04723144364503937|0.014304670131185544|0.018759429417768297|
+------------------+----+-----------------+------------+--------------------+-------------------+--------------------+--------------------+
only showing top 2 rows

In [25]:
#df1.write.mode('overwrite').parquet(outpath+f"/ctr_impression_category_rank/")
(df1.sort(F.col('l1_category_cma_id'), F.col('rank'))
.coalesce(1)
.write.mode('overwrite')
.option("header","true")
.csv(outpath+f"/ctr_impression_category_rank/"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df1=spark.read.parquet(outpath+f"/ctr_impression_category_rank/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:

df1.groupBy('rank').agg(F.sum('impressions_total').alias('impressions_total'),
             F.sum('clicks_total').alias('clicks_total'))\
        .withColumn('ctra', F.col('clicks_total')/F.col('impressions_total'))\
        .withColumn('impression_rate', F.col('impressions_total')/F.sum('impressions_total').over(Window.partitionBy()))\
        .sort(F.col('rank'))\
        .show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+--------------------+--------------------+--------------------+
|rank|impressions_total   |clicks_total        |ctra                |impression_rate     |
+----+--------------------+--------------------+--------------------+--------------------+
|1   |2.099215019767316E8 |1.83762387770202E7  |0.08753862088437746 |0.04112077128201094 |
|2   |1.4395574046403322E8|9176803.22900433    |0.06374739346568203 |0.028198974486235255|
|3   |2.0877354604999998E8|1.445626104285714E7 |0.06924373952720501 |0.04089590230641552 |
|4   |1.3380840638069992E8|5729114.370670995   |0.04281580302489383 |0.026211249550801275|
|5   |1.527004958914865E8 |4437416.832575758   |0.029059609837345374|0.029911953311478647|
|6   |1.1113760267435089E8|2986804.871861472   |0.026874836238939204|0.021770346998134342|
|7   |9.591175431352833E7 |2414116.555952381   |0.025170184543396163|0.01878781008731629 |
|8   |9.147040949166688E7 |2167424.935714286   |0.02369536714396957 |0.017917810954857002|

In [19]:
df1.sort(F.col('l1_category_cma_id'), F.col('rank')).show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----+--------------------+------------------+--------------------+--------------------+---------------------+
|l1_category_cma_id|rank|impressions_total   |clicks_total      |category_sales_total|ctr                 |cvr                  |
+------------------+----+--------------------+------------------+--------------------+--------------------+---------------------+
|1                 |11  |3.4628308E7         |870414.0          |511972.3299999994   |0.02513590903719581 |0.014784791968466937 |
|10                |1   |930538.0            |39268.0           |0.0                 |0.04219924387827257 |0.0                  |
|10                |2   |694386.0            |19947.0           |0.0                 |0.028726097588373037|0.0                  |
|10                |3   |842582.0            |21220.0           |0.0                 |0.025184492429223507|0.0                  |
|10                |4   |923744.0            |20918.0           |0.0                 |0.02

In [29]:
(df1.sort(F.col('l1_category_cma_id'), F.col('rank'))
.coalesce(1)
.write.mode('overwrite')
.option("header","true")
.csv(outpath+f"/ctr_rank/"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# diff

In [20]:
df2=df.withColumn('diff', F.col('impressions_adjust')-F.col('clicks_adjust'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df2=df2.groupBy('l1_category_cma_id', 'rank').agg(F.mean('diff').alias('avg_diff'))\
.sort(F.col('l1_category_cma_id'), F.col('rank'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
(df2.coalesce(1)
.write.mode('overwrite')
.option("header","true")
.csv(outpath+f"/impression_click_diff_rank/"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# impression by category

In [14]:
df3=df.groupBy('l1_category_cma_id').agg(F.sum('impressions_adjust').alias('impressions_total'),
                                        F.sum('clicks_adjust').alias('clicks_total'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df3=df3.withColumn('ratio',  F.col('impressions_total')/F.sum('impressions_total').over(Window.partitionBy()))\
.withColumn('ctr', F.col('clicks_total')/F.col('impressions_total'))\
.withColumn('factor', F.col('ctr')/F.col('ratio'))\
.sort(F.col('factor').desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
(df3.coalesce(1)
.write.mode('overwrite')
.option("header","true")
.csv(outpath+f"/ctr_impression_category/"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df3=spark.read.option("header","true").csv(outpath+f"/ctr_impression_category/").select('l1_category_cma_id', F.col('factor').cast('double'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

60

In [17]:
df3.select('factor').summary().show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+
|summary|factor             |
+-------+-------------------+
|count  |60                 |
|mean   |1032.4085624426339 |
|stddev |3843.0207685230157 |
|min    |0.09137288198098248|
|25%    |0.49548672684804557|
|50%    |1.5338269103131958 |
|75%    |6.644734595901197  |
|max    |19975.113769138818 |
+-------+-------------------+

In [11]:
from pyspark.ml.feature import QuantileDiscretizer


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
discretizer = QuantileDiscretizer(numBuckets=10, inputCol="factor", outputCol="result")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
result = discretizer.fit(df3).transform(df3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
result.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- l1_category_cma_id: string (nullable = true)
 |-- factor: double (nullable = true)
 |-- result: double (nullable = true)

In [24]:
result.show(100,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+------+
|l1_category_cma_id|factor             |result|
+------------------+-------------------+------+
|6810              |19975.113769138818 |9.0   |
|6811              |16222.65244710502  |9.0   |
|6808              |14924.439752180362 |9.0   |
|6816              |5539.278579302361  |9.0   |
|6842              |4425.002680249869  |9.0   |
|442               |284.9789531886355  |9.0   |
|49                |135.60232614491716 |9.0   |
|391               |112.1890982712995  |8.0   |
|6809              |107.761978443923   |8.0   |
|6838              |48.99171682992649  |8.0   |
|6822              |33.693677173323934 |8.0   |
|6826              |30.709089281459907 |8.0   |
|36                |19.020343014111553 |8.0   |
|6839              |7.303432252488425  |7.0   |
|44                |7.07706808776292   |7.0   |
|35                |6.644734595901198  |7.0   |
|60                |5.750736519757597  |7.0   |
|34                |4.981285102856705  |

In [26]:
df3.withColumn("summary",
               F.expr("approx_percentile(factor, array(0.1, 0.2,0.3,0.4,0.5,0.6, 0.7,0.8,0.9,1.0), 1000000)").alias("quantiles")).show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"grouping expressions sequence is empty, and '`l1_category_cma_id`' is not an aggregate function. Wrap '(percentile_approx(`factor`, array(CAST(0.1BD AS DECIMAL(2,1)), CAST(0.2BD AS DECIMAL(2,1)), CAST(0.3BD AS DECIMAL(2,1)), CAST(0.4BD AS DECIMAL(2,1)), CAST(0.5BD AS DECIMAL(2,1)), CAST(0.6BD AS DECIMAL(2,1)), CAST(0.7BD AS DECIMAL(2,1)), CAST(0.8BD AS DECIMAL(2,1)), CAST(0.9BD AS DECIMAL(2,1)), 1.0BD), 1000000) AS `summary`)' in windowing function(s) or wrap '`l1_category_cma_id`' in first() (or first_value) if you don't care which value you get.;;\nAggregate [l1_category_cma_id#1, impressions_total#115, clicks_total#117, ratio#122, ctr#128, factor#134, percentile_approx(factor#134, cast(array(cast(0.1 as decimal(2,1)), cast(0.2 as decimal(2,1)), cast(0.3 as decimal(2,1)), cast(0.4 as decimal(2,1)), cast(0.5 as decimal(2,1)), cast(0.6 as decimal(2,1)), cast(0.7 as decimal(2,1)), cast(0.8 as decimal(2,1)), cast(0.9 as decimal(2,1)), 1.0) as array<double>), 10

In [16]:
df3.show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+---------------------+---------------------+-------------------+
|l1_category_cma_id|impressions_total   |clicks_total      |ratio                |ctr                  |factor             |
+------------------+--------------------+------------------+---------------------+---------------------+-------------------+
|6810              |28709.0             |3225.0            |5.62370320152392E-6  |0.11233411125431049  |19975.113769138818 |
|6811              |30235.0             |2905.0            |5.922625876835686E-6 |0.09608070117413593  |16222.652447105016 |
|6808              |2026.0              |12.0              |3.968658847848222E-7 |0.005923000987166831 |14924.43975218036  |
|6816              |960.0               |1.0               |1.880509621882672E-7 |0.0010416666666666667|5539.27857930236   |
|6842              |45948.0             |1830.0            |9.000589177735939E-6 |0.03982763123530948  |4425.002680249868  |
